# LLM | LLM



LLMアプリケーションで最も一般的な連鎖のタイプは、プロンプトテンプレートをLLMと組み合わせ、オプションで出力パーサーを使用することです。



> The most common type of chaining in any LLM application is combining a prompt template with an LLM and optionally an output parser.



これを行う推奨される方法は、LangChain Expression Languageを使用することです。また、これら3つのコンポーネントを組み合わせる単一のクラスである従来の`LLMChain`のサポートも継続しています。



> The recommended way to do this is using LangChain Expression Language. We also continue to support the legacy `LLMChain`, which is a single class for composing these three components.



## Using LCEL | LCELの使用



`BasePromptTemplate`、`BaseLanguageModel`、そして`BaseOutputParser`はすべて`Runnable`インターフェースを実装しており、互いにパイプ処理されるように設計されているため、LCELの構成が非常に容易になります。



> `BasePromptTemplate`, `BaseLanguageModel` and `BaseOutputParser` all implement the `Runnable` interface and are designed to be piped into one another, making LCEL composition very easy:



In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

prompt = PromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)
runnable = prompt | ChatOpenAI() | StrOutputParser()
runnable.invoke({"product": "colorful socks"})

'VibrantSocks'

インターフェース、組み込み機能、およびクックブックの例についてさらに詳しくは、[LCEL](/docs/expression_language)セクションを参照してください。



> Head to the [LCEL](/docs/expression_language) section for more on the interface, built-in features, and cookbook examples.



## \[Legacy] LLMChain | \[レガシー] LLMチェーン



:::note



これはレガシーなクラスであり、上記の通りLCELの使用が推奨されています。



> This is a legacy class, using LCEL as shown above is preferred.



:::



`LLMChain`は、言語モデルにいくつかの機能を追加するシンプルなチェーンです。LangChain全体で広く使用されており、他のチェーンやエージェントにも組み込まれています。



> An `LLMChain` is a simple chain that adds some functionality around language models. It is used widely throughout LangChain, including in other chains and agents.



`LLMChain`は、`PromptTemplate`と言語モデル（LLMまたはチャットモデルのいずれか）で構成されています。これは、提供された入力キーの値（もし利用可能ならメモリキーの値も含む）を使用してプロンプトテンプレートをフォーマットし、フォーマットされた文字列をLLMに渡して、LLMの出力を返します。



> An `LLMChain` consists of a `PromptTemplate` and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.



### Get started | 始めましょう



In [4]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
llm_chain("colorful socks")

{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

### Additional ways of running `LLMChain` | `LLMChain`を実行する追加の方法



`Chain`オブジェクトに共通する`__call__`メソッドと`run`メソッド以外に、`LLMChain`はチェーンロジックを呼び出す他のいくつかの方法も提供しています：



> Aside from `__call__` and `run` methods shared by all `Chain` object, `LLMChain` offers a few more ways of calling the chain logic:



* `apply` は、入力のリストに対してチェーンを実行することができます：

  > `apply` allows you run the chain against a list of inputs:



In [5]:
input_list = [{"product": "socks"}, {"product": "computer"}, {"product": "shoes"}]
llm_chain.apply(input_list)

[{'text': '\n\nSocktastic!'},
 {'text': '\n\nTechCore Solutions.'},
 {'text': '\n\nFootwear Factory.'}]

* `generate`は`apply`に似ていますが、文字列ではなく`LLMResult`を返すという点が異なります。`LLMResult`には、トークン使用量や終了理由など、有益な生成情報が含まれていることがしばしばあります。

  > `generate` is similar to `apply`, except it return an `LLMResult` instead of string. `LLMResult` often contains useful generation such as token usages and finish reason.



In [6]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSocktastic!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nTechCore Solutions.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nFootwear Factory.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 36, 'total_tokens': 55}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('9a423a43-6d35-4e8f-9aca-cacfc8e0dc49')), RunInfo(run_id=UUID('a879c077-b521-461c-8f29-ba63adfc327c')), RunInfo(run_id=UUID('40b892fa-e8c2-47d0-a309-4f7a4ed5b64a'))])

* `predict`メソッドは、入力キーをPythonの辞書ではなくキーワード引数として指定する点を除いて、`run`メソッドと似ています。

  > `predict` is similar to `run` method except that the input keys are specified as keyword arguments instead of a Python dict.



In [7]:
# Single input example
llm_chain.predict(product="colorful socks")

'\n\nSocktastic!'

In [8]:
# Multiple inputs example
template = """Tell me a {adjective} joke about {subject}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0))

llm_chain.predict(adjective="sad", subject="ducks")

'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'

### Parsing the outputs | 出力の解析



`LLMChain`は、基礎となる`prompt`オブジェクトに出力パーサーがあっても、デフォルトでは出力を解析しません。LLMの出力にその出力パーサーを適用したい場合は、`predict`の代わりに`predict_and_parse`を、`apply`の代わりに`apply_and_parse`を使用してください。



> By default, `LLMChain` does not parse the output even if the underlying `prompt` object has an output parser. If you would like to apply that output parser on the LLM output, use `predict_and_parse` instead of `predict` and `apply_and_parse` instead of `apply`.



`predict`を使用して：



> With `predict`:



In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all the colors in a rainbow"""
prompt = PromptTemplate(
    template=template, input_variables=[], output_parser=output_parser
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict()

'\n\nRed, orange, yellow, green, blue, indigo, violet'

`predict_and_parse`を使用して：



> With `predict_and_parse`:



In [10]:
llm_chain.predict_and_parse()

/Users/bagatur/langchain/libs/langchain/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['Red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

### Initialize from string | 文字列から初期化する



文字列テンプレートから直接`LLMChain`を構築することもできます。



> You can also construct an `LLMChain` from a string template directly.



In [11]:
template = """Tell me a {adjective} joke about {subject}."""
llm_chain = LLMChain.from_string(llm=llm, template=template)

In [12]:
llm_chain.predict(adjective="sad", subject="ducks")

'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'